do everything up untill merge from dsa21


In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, precision_score, precision_recall_curve, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer


In [2]:
#Import the data 
# read all tables 
customer_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [3]:
# deal with gender null 
# deal with age null
customer_final.dropna(subset=['gender', 'age'], inplace=True)

In [4]:
customer_final['join_date'] = pd.to_datetime(customer_final['join_date'])
customer_final['last_purchase_date'] = pd.to_datetime(customer_final['last_purchase_date'])
transactions_final['transaction_date'] = pd.to_datetime(transactions_final['transaction_date'])
marketing_final['campaign_date'] = pd.to_datetime(marketing_final['campaign_date'])

feature engineering 

In [5]:
#create new database to hold CLV
clv_final = transactions_final.groupby('customer_id')['transaction_amount'].sum().reset_index()
clv_final.rename(columns={'transaction_amount': 'clv'}, inplace=True)


In [6]:
clv_final

,customer_id,clv
0,1,3509.48
1,2,6081.32
2,3,1454.87
3,4,7874.68
4,5,15524.55
...,...,...
9995,9996,5498.20
9996,9997,5848.30
9997,9998,3503.13
9998,9999,6721.86


above is doing 
Grouping Transactions: The code groups all the transaction records by each customer_id.
Calculating Total Spend: For each customer, it calculates the total spend by summing up the transaction_amount.
Creating New DataFrame: The result is a new DataFrame clv_df with columns customer_id and transaction_amount.
Renaming for Clarity: The transaction_amount column is renamed to clv to represent Customer Lifetime Value.

In [7]:
customer_final['customer_lifetime'] = (customer_final['last_purchase_date'] - customer_final['join_date']).dt.days

In [8]:
customer_final

,customer_id,join_date,last_purchase_date,age,gender,location,customer_lifetime
0,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,118
3,4,2022-01-01,2022-09-01,29.0,Male,Grossstad,243
5,6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,405
7,8,2023-02-09,2023-08-09,68.0,Male,Marquezton,181
8,9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,568
...,...,...,...,...,...,...,...
9995,9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,240
9996,9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,200
9997,9998,2023-09-17,2024-01-30,39.0,Male,New John,135
9998,9999,2022-05-10,2022-07-15,31.0,Female,Andrewland,66


this is within customer final database is doing 
Creating the Column: A new column named customer_lifetime is added to the customers_df DataFrame.
Calculating Date Difference: For each customer, the code calculates the time difference between their last_purchase_date and their join_date.
Converting to Days: The time difference is then converted from a time duration to the number of days.


In [9]:
merged_final = customer_final.merge(clv_final, on='customer_id', how='left')

In [10]:
merged_final

,customer_id,join_date,last_purchase_date,age,gender,location,customer_lifetime,clv
0,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,118,3509.48
1,4,2022-01-01,2022-09-01,29.0,Male,Grossstad,243,7874.68
2,6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,405,10254.44
3,8,2023-02-09,2023-08-09,68.0,Male,Marquezton,181,11710.62
4,9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,568,10382.96
...,...,...,...,...,...,...,...,...
8517,9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,240,5498.20
8518,9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,200,5848.30
8519,9998,2023-09-17,2024-01-30,39.0,Male,New John,135,3503.13
8520,9999,2022-05-10,2022-07-15,31.0,Female,Andrewland,66,6721.86


The result of this operation is a new DataFrame merged_df that contains all the columns from customers_df along with the clv column from clv_df.

In [11]:
# agg --> transaction, engagment, and marketing by customer ID 
engagement_agg = engagement_final.groupby('customer_id').sum()
marketing_agg = marketing_final.groupby('customer_id').size()
transactions_aggregreg = transactions_final.groupby('customer_id').count

In [12]:
transactions_aggregreg.head()

AttributeError: 'function' object has no attribute 'head'